In [1]:
import requests
import time
import json
from IPython.display import JSON

In [2]:
va=[]
i=0
count=0
baseurl='https://api.openalex.org/sources?filter=is_in_doaj:true&mailto=restrepo@udea.edu.co&per-page=200'
nc='*'

In [3]:
while nc:
    i+=1
    url=baseurl+f'&cursor={nc}'
    r=requests.get(url)
    cotmp=r.json().get('results')

    if i==1:
        count=r.json().get('meta').get('count')
    nc=r.json().get('meta').get('next_cursor')
    #progress
    print(f'{i} → {200*i}/{count}',end='\r')
    #Append to dictionary list (WARNING: HIG RAM USAGE)
    for p in cotmp:
        va.append(p)
    #API limits
    time.sleep(0.1)
    #EMERGENGY EXIT
    if i*200>count:
        break

In [239]:
f=open('/scratch/restrepo/oa.json','w')
json.dump(va,f)
f.close()

In [267]:
ISSN_OpenAlex = False

In [268]:
df=pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vRqkKT0ovQFEHTEKJjEGmX_3wpmiDH-wVqucnfYK9P_pOpolF8SpdlLXcpM3PwjFv2QlQbCW6CSFJes/pub?output=csv')

In [291]:
df.shape

(20061, 54)

In [269]:
if ISSN_OpenAlex:
    issns=[d.get('issn_l') for d in va]
else:    
    issns = df.apply(lambda d: [x for x in [d.get('Journal ISSN (print version)'),d.get('Journal EISSN (online version)')] 
                                 if isinstance(x,str) ][0] 
         , axis='columns' ).to_list()

In [270]:
len(issns)

20061

In [282]:
doaj=[]
i=0
baseurl='https://doaj.org/api/v3/search/journals/issn:'

In [283]:
missing_issns=issns.copy()
failed_issns=[]

In [284]:
unchecked_issns=[]
not_found=[]

In [292]:
#ISSNS=issns
ISSNS=missing_issns

In [288]:
#issn=issns[0]
#url=baseurl+f'{issn}'
#r=requests.get(url)

In [290]:
len(issns)

20061

In [ ]:
#END when all missings are in failed
for issn in ISSNS:
    i+=1
    url=baseurl+f'{issn}'
    try:
        r=requests.get(url)
    except:
        unchecked_issns.append(issn)
        continue
    if r.status_code == 200 and r.json().get('results'):
        doaj.append(r.json().get('results')[0])
        missing_issns.remove(issn)
    else:
        if r.status_code == 200 and r.json().get('total')==0:
            not_found.append(issn)
        else:
            failed_issns.append(issn)
    #progress
    print(f'{str(i).zfill(5)} → {str( len(doaj) ).zfill(5)}:{str( len(not_found) ).zfill(5)}:{str( len(missing_issns) ).zfill(5)}:{str( len(failed_issns) ).zfill(5)}→ {len(issns)}',end='\r')
    time.sleep(2)
    #if i==20:
    #    break

In [281]:
'2688-8130'

['0146-0404',
 '1660-4601',
 '0018-5345',
 '0036-7672',
 '2314-6133',
 '1024-123X',
 '0368-4814',
 '2041-6520',
 '1741-427X',
 '0160-9289',
 '0934-0874',
 '0950-2688',
 '1729-3774']

In [127]:
f=open('/scratch/restrepo/doaj.json','w')
json.dump(doaj,f)
f.close()

In [128]:
ls -lh /scratch/restrepo/doaj.json 

-rw-r--r-- 1 restrepo restrepo 29M Oct 30  2023 /scratch/restrepo/doaj.json


In [124]:
failed_issns=list(set(failed_issns))
len(failed_issns),failed_issns[:2]

(1254, ['2504-5687', '2182-6951'])

In [125]:
len(set(missing_issns).difference(failed_issns))

0

In [252]:
f=open('/scratch/restrepo/doaj.json','r')
doaj=json.load(f)
f.close()

In [254]:
df=pd.read_json('/scratch/restrepo/doaj.json')

In [264]:
issns_doaj=df.bibjson.apply(lambda d: [ x for x in [d.get('eissn'),d.get('pissn')] #if isinstance(x,str)
                                      ]).to_list()

In [265]:
issns_doaj=[item for sublist in issns_doaj for item in sublist]

In [258]:
len(issns_doaj)

13134

In [196]:
expected_doaj=list(set(issns).difference(failed_issns))

In [197]:
len(expected_doaj)

13123

In [198]:
df.shape

(13134, 5)

In [266]:
len(set(issns).difference(issns_doaj))

1249

In [162]:
list(set(issns).difference(    [item for sublist in issns_doaj for item in sublist] ))[0]

'2504-5687'

In [ ]:
'2504-5687' in issns

In [153]:
df.shape

(13134, 5)

In [155]:
len(set(issns_doaj).difference(issns))

884

In [112]:
len(set(failed_issns)), len(missing_issns), len(issns),len(missing_issns)-len(issns)

(1245, 1317, 14383, -13066)

In [72]:
len(doaj)+len(missing_issns)

14383

In [66]:
r=requests.get(url)

In [83]:
r.json()

{'total': 1,
 'page': 1,
 'pageSize': 10,
 'timestamp': '2023-10-29T09:35:14.067701Z',
 'query': 'issn:2154-8935',
 'results': [{'id': '8e60944cdf9b47c1b3c16df9206a6fb2',
   'created_date': '2020-06-05T08:37:37Z',
   'last_updated': '2023-03-29T11:36:00Z',
   'bibjson': {'boai': True,
    'eissn': '2154-8935',
    'publication_time_weeks': 33,
    'title': 'Journal of Public and Professional Sociology',
    'oa_start': 2005,
    'apc': {'has_apc': False,
     'url': 'https://digitalcommons.kennesaw.edu/jpps/about.html'},
    'article': {'license_display': ['No']},
    'copyright': {'author_retains': False},
    'deposit_policy': {'has_policy': False},
    'editorial': {'review_url': 'https://digitalcommons.kennesaw.edu/jpps/about.html',
     'board_url': 'https://digitalcommons.kennesaw.edu/jpps/editorialboard.html',
     'review_process': ['Anonymous peer review']},
    'institution': {'name': 'Georgia Sociological Association'},
    'other_charges': {'has_other_charges': False,
     

In [63]:
len(missing_issns)-len(issns)

-65

In [37]:
import pandas as pd

In [45]:
pd.DataFrame(doaj)

,id,created_date,last_updated,bibjson,admin
0,2fdf1470373343b7bd4f825179c685f5,2007-03-30T10:53:19Z,2023-03-29T11:34:49Z,"{'alternative_title': 'PLOS ONE', 'boai': True...","{'seal': True, 'ticked': True}"
1,2fdf1470373343b7bd4f825179c685f5,2007-03-30T10:53:19Z,2023-03-29T11:34:49Z,"{'alternative_title': 'PLOS ONE', 'boai': True...","{'seal': True, 'ticked': True}"
2,2fdf1470373343b7bd4f825179c685f5,2007-03-30T10:53:19Z,2023-03-29T11:34:49Z,"{'alternative_title': 'PLOS ONE', 'boai': True...","{'seal': True, 'ticked': True}"
3,2fdf1470373343b7bd4f825179c685f5,2007-03-30T10:53:19Z,2023-03-29T11:34:49Z,"{'alternative_title': 'PLOS ONE', 'boai': True...","{'seal': True, 'ticked': True}"
4,2fdf1470373343b7bd4f825179c685f5,2007-03-30T10:53:19Z,2023-03-29T11:34:49Z,"{'alternative_title': 'PLOS ONE', 'boai': True...","{'seal': True, 'ticked': True}"
5,2fdf1470373343b7bd4f825179c685f5,2007-03-30T10:53:19Z,2023-03-29T11:34:49Z,"{'alternative_title': 'PLOS ONE', 'boai': True...","{'seal': True, 'ticked': True}"
6,75844ed25a2249039bd9c32da9b5f22b,2021-02-15T11:28:03Z,2023-04-12T14:49:04Z,"{'boai': True, 'eissn': '1083-351X', 'pissn': ...","{'seal': False, 'ticked': True}"
7,a93400557c92470b8603e96796297aab,2011-11-04T11:48:02Z,2023-04-12T15:11:06Z,"{'boai': True, 'eissn': '2045-2322', 'publicat...","{'seal': False, 'ticked': True}"
8,af8e2cd0d2e94fba997a416d65b8a965,2018-01-22T11:54:20Z,2023-08-01T15:04:19Z,"{'boai': True, 'eissn': '2046-2069', 'publicat...","{'seal': False, 'ticked': True}"
9,6e6cfb34e9c1483f9b1bef353b647ec8,2003-11-20T00:00:00Z,2023-07-07T16:31:47Z,"{'alternative_title': 'IJMS', 'boai': True, 'e...","{'seal': True, 'ticked': True}"


In [28]:
f=open('/scratch/restrepo/doaj.json','w')
json.dump(doaj,f)
f.close()

In [20]:
r.status_code

200

In [22]:
JSON(r.json().get('results'))

<IPython.core.display.JSON object>